In [1]:
import pathlib
import warnings

import numpy as np
import pandas as pd
import shapely

warnings.filterwarnings("ignore")

In [2]:
from pedpy import WalkableArea

walkable_area = WalkableArea(
    # complete area
    [
        (3.5, -2),
        (3.5, 8),
        (-3.5, 8),
        (-3.5, -2),
    ],
    obstacles=[
        # left barrier
        [
            (-0.7, -1.1),
            (-0.25, -1.1),
            (-0.25, -0.15),
            (-0.4, 0.0),
            (-2.8, 0.0),
            (-2.8, 6.7),
            (-3.05, 6.7),
            (-3.05, -0.3),
            (-0.7, -0.3),
            (-0.7, -1.0),
        ],
        # right barrier
        [
            (0.25, -1.1),
            (0.7, -1.1),
            (0.7, -0.3),
            (3.05, -0.3),
            (3.05, 6.7),
            (2.8, 6.7),
            (2.8, 0.0),
            (0.4, 0.0),
            (0.25, -0.15),
            (0.25, -1.1),
        ],
    ],
)

In [ ]:
import matplotlib.pyplot as plt

from pedpy import plot_walkable_area

plot_walkable_area(walkable_area=walkable_area).set_aspect("equal")
plt.show()

In [4]:
from pedpy import MeasurementArea

measurement_area = MeasurementArea(
    [(-0.4, 0.5), (0.4, 0.5), (0.4, 1.3), (-0.4, 1.3)]
)

In [ ]:
import matplotlib.pyplot as plt

from pedpy import plot_measurement_setup

plot_measurement_setup(
    walkable_area=walkable_area,
    measurement_areas=[measurement_area],
    ma_line_width=2,
    ma_alpha=0.2,
).set_aspect("equal")
plt.show()

In [ ]:
from pedpy import TrajectoryData, load_trajectory, plot_trajectories
from pedpy.column_identifier import FRAME_COL

traj_total = load_trajectory(
    trajectory_file=pathlib.Path("demo-data/bottleneck/040_c_56_h-.txt")
)
min_frame = 300
max_frame = 310

traj_cut = TrajectoryData(
    data=traj_total.data[
        (traj_total.data[FRAME_COL] > min_frame)
        & (traj_total.data[FRAME_COL] < max_frame)
    ],
    frame_rate=traj_total.frame_rate,
)

plot_trajectories(
    traj=traj_cut, traj_alpha=0.5, traj_width=1, walkable_area=walkable_area
).set_aspect("equal")
plt.show()

The Fast-Marching-Method is a way to compute the propagation of wavefronts. In this Notebook it is explained how to use this method to compute individual area for pedestrian.

The Fast-Marching-Method discretizes the room into squares of equal size and computes the propagation of wavefronts on this grid.
For each pedestrian a wavefront is started at the same time. All the grid-cells that the wavefront reaches will be added to the individual area. The Propagation of the wavefront stops if it reaches a cell that has already been assigned to another person, a cell that is outside the walkable area, or the Cutoff-radius is extended. In this way, individual areas can be created that are similar to the Voronoi cells, but take into account the walkable area.

There are two main steps to computing the individual areas with the Fast-Marching-Method:

At first you need to create a DataFrame containing information about the Shape and the Speedmap.
The shape of each pedestrian is transferred to the grid and is the starting position for the wavefront associated with it.

The Speedmap is an array which specifies the propagation speed of the wavefront.

The GridInfo-object saves data related to the grid. this includes the walkable_area as well as the size of the grid-cells.

It is possible to compute the DataFrame on your own, but you can also use the compute_shape_and_speedmap function.
This function will take the trajectory information, information about the Grid and additional information required to compute the shape and speedmap.
additionaly you need to specify a lambda-function used to create the shape and a lambda-function to create the speedmap.

At first the shape of the Pedestrian is specified as the point of his location from the trajectory. 
The Speedmap will be computed with the default parameter of the function compute_speedmap.

These lambda-function receive the trajectories, information about the cell size and additional information. 
Here no additional information is needed for the computation.

In [ ]:
from pedpy.methods.method_utils import (
    GridInfo,
    apply_computed_speedmap,
    apply_point_as_shape,
    compute_shape_and_speedmap,
)

grid = GridInfo(walkable_area=walkable_area, cell_size=0.05)

shape_and_speedmap = compute_shape_and_speedmap(
    traj=traj_cut,
    grid=grid,
    calc_shape_from_frame=apply_point_as_shape,
    calc_speedmap_from_frame=apply_computed_speedmap,
    additions=None,
)

print(
    f"columns in shape and speedmap dataframe: {list(shape_and_speedmap.columns)}"
)

With the shape_and_speedmap DataFrame it is possible to compute the individual areas with the Fast-Marching-Method.

The result is similar the the result of compute_individual_voronoi_cells and includes the columns id, frame, polygon, density.
Thus the results can be used the same way you can use individual voronoi cells.

In [ ]:
from pedpy.methods.method_utils import compute_individual_areas

individual_areas = compute_individual_areas(
    shape_and_speedmap=shape_and_speedmap, grid=grid, cutoff_distance=1.5
)

print(individual_areas)

Below is a plot with the created individual areas of frame 305. The result are similar to the Voronoi-Cells. However you can see how the wavefront of the person leaving the bottleneck propagate around the non-walkable-area taking into account where a person can walk and where there are obstacles in the way. 

In [ ]:
import matplotlib.pyplot as plt

from pedpy import DENSITY_COL, plot_voronoi_cells

plot_voronoi_cells(
    voronoi_data=individual_areas,
    traj_data=traj_cut,
    frame=305,
    walkable_area=walkable_area,
    color_by_column=DENSITY_COL,
    vmin=0,
    vmax=10,
).set_aspect("equal")
plt.show()

Next, the shape of the pedestrian should not be specified by a point, but by the approximation of the body ellipse of the GCFM.

To do this you can use the compute_shape_and_speedmap together with the lambda-function apply_ellipses_as_shape.
the additional DataFrame has to include the information about velocity in x and y direction as well as the speed of the pedestrian.
These information can be computed by using the compute_individual_speed function with the option compute_velocity=True.
Additionally parameters for the Ellipse of the GCFM can be specified. Otherwise the default parameters of GCFM are used.


In [10]:
from pedpy import SpeedCalculation, compute_individual_speed
from pedpy.column_identifier import (
    FRAME_COL,
    ID_COL,
    SPEED_COL,
    V_X_COL,
    V_Y_COL,
)
from pedpy.methods.method_utils import (
    GridInfo,
    apply_computed_speedmap,
    apply_ellipses_as_shape,
    compute_shape_and_speedmap,
)

grid = GridInfo(walkable_area=walkable_area, cell_size=0.05)

individual_speed = compute_individual_speed(
    traj_data=traj_total,
    frame_step=10,
    compute_velocity=True,
    speed_calculation=SpeedCalculation.BORDER_SINGLE_SIDED,
)
additions = individual_speed[[ID_COL, FRAME_COL, SPEED_COL, V_X_COL, V_Y_COL]]

additions["a_v"] = 0.5
additions["a_min"] = 0.1
additions["b_min"] = 0.1
additions["b_max"] = 0.2

shape_and_speedmap = compute_shape_and_speedmap(
    traj=traj_cut,
    grid=grid,
    calc_shape_from_frame=apply_ellipses_as_shape,
    calc_speedmap_from_frame=apply_computed_speedmap,
    additions=additions,
)

In [11]:
individual_areas = compute_individual_areas(
    shape_and_speedmap=shape_and_speedmap, grid=grid, cutoff_distance=1.5
)

In [ ]:
plot_voronoi_cells(
    voronoi_data=individual_areas,
    traj_data=traj_cut,
    frame=305,
    walkable_area=walkable_area,
    color_by_column=DENSITY_COL,
    vmin=0,
    vmax=10,
).set_aspect("equal")

# Here the shape is plotted by plot_voronoi_cells however this is not compatible with Multipolygons.
shape_and_speedmap["polygon"] = shape_and_speedmap["shape"]
plot_voronoi_cells(voronoi_data=shape_and_speedmap, frame=305).set_aspect(
    "equal"
)

plt.show()